In [1]:
import preprocess as pp
import configs
import persister
import relation
import lda

import re
import json

from stanfordcorenlp import StanfordCoreNLP
from nltk import sent_tokenize, word_tokenize, pos_tag
from nltk.corpus import wordnet
from nltk.tree import Tree
from imp import reload

not use stanford CoreNLP client!


In [ ]:
# 文档向量化测试
# count vectorizer

In [ ]:
a = "Many existing knowledge bases(KBs), including Freebase, Yago, and NELL, rely on a ﬁxed ontology, given as an input to the system, which deﬁnes the data to be cataloged in the KB, i.e., a hierarchy of categories and relations between them. The system then extracts facts that match the predeﬁned ontology. We propose an unsupervised model that jointly learns a latent ontological structure of an input corpus, and identiﬁes facts from the corpus that match the learned structure. Our approach combines mixed membership stochastic block models and topic models to infer a structure by jointly modeling text, a latent concept hierarchy, and latent semantic relationships among the entities mentioned in the text. As a case study, we apply the model to a corpus of Web documents from the software domain,and evaluate the accuracy of the various components of the learned ontology. "
b = ["hello world. what the fuck."]

In [ ]:
c = [" ".join(preprocess(a))]
vector = CountVectorizer(ngram_range=(1, 2), vocabulary=["knowledge base"], stop_words='english')
vector.build_analyzer()
x = vector.fit_transform(c)

In [ ]:
estimator = CountVectorizer(ngram_range=(1, 2), vocabulary=['fuck', 'hello', 'world', "hello world"], stop_words='english')
estimator.build_analyzer()
res = estimator.fit_transform(b)

In [ ]:
estimator.get_feature_names()

In [ ]:
res.toarray()
# estimator.vocabulary_

In [ ]:
# 斯坦福nlp工具测试
# stanfordcorenlp

In [ ]:
nlp = StanfordCoreNLP('/mnt/d/stanford-corenlp-full-2018-02-27', port=9000)
sentence = 'Brink is taking part in the final exercise.'
print('Tokenize:', nlp.word_tokenize(sentence))
print('Part of Speech:', nlp.pos_tag(sentence))
print('Named Entities:', nlp.ner(sentence))
print('Constituency Parsing:', nlp.parse(sentence))
print('Dependency Parsing:', nlp.dependency_parse(sentence))

nlp.close() # Do not forget to close! The backend server will consume a lot memery.

In [ ]:
# 官方corenlp api
# stanza

In [ ]:
# 新闻数据集测试
# 20newsgroups

In [2]:
import newsgroups
size = 500
tmp = newsgroups.get_news_data(size)
newsdata = []
for cate in tmp:
    newsdata.extend(cate)

In [13]:
for i in [" ".join([w["originalText"] for w in sent["tokens"]]) for sent in res["sentences"]]:
    print(i)

In regards to fractal commpression , I have seen 2 fractal compressed " movies " .
They were both fairly impressive .
The first one was a 64 gray scale " movie " of Casablanca , it was 1.3 MB and had 11 minutes of 13 fps video .
It was a little grainy but not bad at all .
The second one I saw was only 3 minutes but it had 8 bit color with 10 fps and measured in at 1.2 MB .
I consider the fractal movies a practical thing to explore .
But unlike many other formats out there , you do end up losing resolution .
I do n't know what kind of software/hardware was used for creating the " movies " I saw but the guy that showed them to me said it took 5-15 minutes per frame to generate .
But as I said above playback was 10 or more frames per second .
And how else could you put 11 minutes on one floppy disk ?


In [21]:
from stanza.server import CoreNLPClient
with CoreNLPClient(properties="./corenlp_server.props", timeout=30000, memory='5G') as client:
    res = relation.corenlp_annotate(client, pp.format_news(newsdata[0]))

Setting server defaults from: ./corenlp_server.props
Starting server with command: java -Xmx5G -cp /mnt/d/stanford-corenlp-full-2018-02-27/* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 30000 -threads 5 -maxCharLength 100000 -quiet True -serverProperties ./corenlp_server.props -preload pos,parse,depparse,lemma


In [ ]:
texts = [' '.join(pp.preprocess_abstract(a))
persister.save_json(configs.NEWSDATA, texts)

In [ ]:
# 处理成句子
# nltk.sent_tokenize

In [ ]:
newssent = []
for news in newsdata:
    newssent.append(pp.split2sent(news))
persister.save_json(configs.NEWSSENT, newssent)

In [ ]:
newssenttoken = []
for news in newssent:
    tokenized_lemmatized_news = []
    for sent in news:
        tokenized_lemmatized_news.append(relation.lemmatize_sent_words(sent))
    newssenttoken.append(tokenized_lemmatized_news)
persister.save_json(configs.NEWSSENTTOKEN, newssenttoken)

In [ ]:
# load preprossed data

In [2]:
rawnews = persister.load_json(configs.RAWNEWS)
newsparse = persister.read_parse()

In [9]:
print(rawnews[36])

I have posted a DOS MPEG decoder/player to alt.binaries.pictures.utilities.

Here is a short description and some technical information, taken from the
accompanying documentation:


                              DMPEG V1.0

                       Public Domain MPEG decoder

                           by Stefan Eckart


0. Features

DMPEG/DMPLAY is another MPEG decoder/player for the PC:


 - decodes (nearly) the full MPEG video standard
   (I,P,B frames, frame size up to at least 352x240 supported)

 - saves decoded sequence in 8 or 24bit raw file for later display

 - optional on-screen display during decoding (requires VGA)

 - several dithering options: ordered dither, Floyd-Steinberg, grayscale

 - color-space selection

 - runs under DOS, 640KB RAM, no MS-Windows required

 - very compact (small code / small data models, 16 bit arithmetic)

 - real time display of the raw file by a separate player for
   VGA and many Super-VGAs

...

4. Technical information

The player is a rathe

In [19]:
# load lda res
terms, doc_topic, topic_word = persister.read_lda(configs.NEWSLDA)
lda.print_topics(topic_word, terms, doc_topic)
top_terms, top_docs = lda.get_topics(topic_word, terms, doc_topic)

FileNotFoundError: [Errno 2] No such file or directory: 'news_lda.npz'

In [12]:
idx_news = 1195
idx_sent = 40

In [16]:
rawnews[36]

"I have posted a DOS MPEG decoder/player to alt.binaries.pictures.utilities.\n\nHere is a short description and some technical information, taken from the\naccompanying documentation:\n\n\n                              DMPEG V1.0\n\n                       Public Domain MPEG decoder\n\n                           by Stefan Eckart\n\n\n0. Features\n===========\n\nDMPEG/DMPLAY is another MPEG decoder/player for the PC:\n\n\n - decodes (nearly) the full MPEG video standard\n   (I,P,B frames, frame size up to at least 352x240 supported)\n\n - saves decoded sequence in 8 or 24bit raw file for later display\n\n - optional on-screen display during decoding (requires VGA)\n\n - several dithering options: ordered dither, Floyd-Steinberg, grayscale\n\n - color-space selection\n\n - runs under DOS, 640KB RAM, no MS-Windows required\n\n - very compact (small code / small data models, 16 bit arithmetic)\n\n - real time display of the raw file by a separate player for\n   VGA and many Super-VGAs\n\n..

In [23]:
[" ".join([w["lemmma"] for w in sent["tokens"]]) for sent in newsparse[1200]["sentences"]]

['in <smn42b1w165w@cybernet.cse.fau.edu> vlasis@cybernet.cse.fau.edu -lrb- vlasis theodore -rrb- write : > tobias@convex.com -lrb- Allen Tobias -rrb- write : >> in article <1993apr15.024246.8076@virginia.edu> ejv2j@virginia.edu -lrb- `` Erik Vel >> > this happen about a year ago on the Washington DC Beltway .',
 '>> > snot nosed drunken kid decide it would be really cool to >> > throw huge rock down on car from a overpass .',
 'four or five >> > car be hit .',
 'there be several serious injury , and sadly >> > a small girl sit in the front seat of one of they be strike >> > in the head by one of the larger rock .',
 'I do not recall if she >> > make it , but I think she be comatose for a month or so and >> > doctor be not hold out hope that she would live .',
 '>> > >> > what the hell be happen to this great country of ours ?',
 'I >> > can see boyhood prank of pee off of bridge and such , but >> > 20 pound rock ??!',
 'have we society really stoop this low ??',
 '>> > >> > Erik velapo

In [ ]:
print(pos_tag(word_tokenize(newssent[idx_news][43])))
print(pp.WNL.lemmatize("has", wordnet.VERB))

In [ ]:
print(len(newsparse[idx_news]["sentences"]))
print(newsparse[idx_news]["sentences"][idx_sent].keys())
examparse = newsparse[idx_news]["sentences"][idx_sent]
print(examparse["enhancedPlusPlusDependencies"])

In [ ]:
tt = Tree.fromstring(examparse["parse"])
tt.pretty_print()

In [ ]:
t4 = [t[0] for t in top_terms[3]]
t4_values = [t[1] for t in top_terms[3]]
print(t4)
print(t4_values)
idxs, importance, count = relation.extract_important_sents(newssenttoken[idx_news], t4, t4_values)
print(idxs, importance[40], count[40])

In [ ]:
relation.extract_word_relation_from_sent(i, newsparse[1200]["sentences"][sent_idx]["enhancedDependencies"])

In [ ]:
for term in top_terms[3]:
    for news in top_docs[3]:
        time.sleep(2)
        news_idx = news[0]
        for sent_idx,sent in enumerate(newssenttoken[news_idx]):
            if type(newsparse[news_idx]) is str:
                print(news_idx, "parse", "err")
                break
            topic_word_idxs = relation.get_word_idx(term[0], sent)
            for i in topic_word_idxs:
                print(sent_idx, i, news_idx)
                rs = relation.extract_word_relation_from_sent(i, newsparse[news_idx]["sentences"][sent_idx]["enhancedDependencies"])
                for r in rs:
                    print(relation.convert_relation2str(r))

In [ ]:
t = Tree.fromstring(newsparse[idx_news]["sentences"][5]["parse"])
t.pretty_print()

In [ ]:
# test example res

In [ ]:
import json
with open("exampleres.json",encoding="utf8")as f:
    res = json.load(f)
print(res)